## Extracción: Montos Programación del monto de inversión (S/) del PMI

In [ ]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ================================================================
# CONFIGURACIÓN OPTIMIZADA
# ================================================================
RUTA_ENTRADA = r"C:\Users\user\Downloads\mef1.xlsx"
RUTA_SALIDA = r"C:\Users\user\Downloads\CUI DATOS PMI Y OPMI.xlsx"
MAX_REINTENTOS = 2
MODO_VISIBLE = True
GUARDAR_CADA = 5

TIMEOUT_PAGINA = 20
TIMEOUT_ELEMENTO = 10

# ================================================================
# FUNCIONES DE CHECKPOINT
# ================================================================
def cargar_progreso():
    """Carga progreso previo si existe"""
    if os.path.exists(RUTA_SALIDA):
        try:
            df = pd.read_excel(RUTA_SALIDA)
            print(f"📥 Progreso encontrado: {len(df)} CUIs ya procesados")
            return df.to_dict('records')
        except:
            pass
    return []

def obtener_pendientes(completa, procesados):
    """Calcula SOLO CUIs nunca procesados (NO reintentar NO DISPONIBLE)"""
    if not procesados:
        print(f"📋 Todos los CUIs pendientes: {len(completa)}")
        return completa, []
    
    procesados_dict = {str(r['CUI']): r for r in procesados}
    cuis_nuevos = [cui for cui in completa if str(cui) not in procesados_dict]
    
    if cuis_nuevos:
        print(f"📋 CUIs pendientes: {len(cuis_nuevos)}")
    
    return cuis_nuevos, procesados_dict

def guardar(resultados):
    """Guarda progreso"""
    try:
        pd.DataFrame(resultados).to_excel(RUTA_SALIDA, index=False)
    except Exception as e:
        print(f"⚠️ Error guardando: {e}")

# ================================================================
# LEER CUIs
# ================================================================
print("📂 Cargando CUIs...")
df_cui = pd.read_excel(RUTA_ENTRADA)
lista_completa = df_cui['CUI'].astype(str).tolist()

resultados_previos = cargar_progreso()
lista_cuis, procesados_dict = obtener_pendientes(lista_completa, resultados_previos)

if not lista_cuis:
    print("🎉 ¡Todos los CUIs ya fueron procesados!")
    exit()

resultados = resultados_previos.copy()

# ================================================================
# CONFIGURAR NAVEGADOR OPTIMIZADO
# ================================================================
def crear_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()

    if not MODO_VISIBLE:
        options.add_argument("--headless=new")
    else:
        options.add_argument("--start-maximized")

    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-logging")
    options.add_argument("--log-level=3")
    options.add_argument("--silent")

    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.media_stream": 2,
        "profile.default_content_setting_values.geolocation": 2,
    }
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.page_load_strategy = 'normal'

    driver = Chrome(service=service, options=options)
    driver.set_page_load_timeout(TIMEOUT_PAGINA)
    driver.set_script_timeout(15)
    return driver

driver = crear_driver()
print(f"⚡ Procesando {len(lista_cuis)} CUIs pendientes\n")

# ================================================================
# VERIFICAR SI CUI EXISTE EN PMI
# ================================================================
def cui_existe_en_pmi(driver):
    """Verifica si la página muestra un CUI válido o un error"""
    try:
        page_text = driver.find_element(By.TAG_NAME, "body").text.lower()
        
        errores = [
            "no se encontró",
            "no existe",
            "no hay información",
            "sin registros",
            "código único de inversión no válido",
            "cui no válido"
        ]
        
        for error in errores:
            if error in page_text:
                return False
        
        if len(page_text.strip()) < 100:
            return False
        
        try:
            # Verificar que exista la tabla #tblResultado con datos
            tabla = driver.find_element(By.ID, "tblResultado")
            tbody = tabla.find_element(By.TAG_NAME, "tbody")
            filas = tbody.find_elements(By.TAG_NAME, "tr")
            
            if len(filas) == 0:
                return False
                
        except:
            return False
        
        return True
        
    except Exception as e:
        return False

# ================================================================
# EXTRACCIÓN DE DATOS PMI - BASADO EN HTML REAL
# ================================================================
def extraer_datos_pmi(driver):
    """
    Extrae datos de la tabla #tblResultado según estructura HTML:
    Columna 4 (índice 3): OPMI
    Columna 13 (índice 12): Monto año 2026
    Columna 14 (índice 13): Monto año 2027
    Columna 15 (índice 14): Monto año 2028
    Columna 16 (índice 15): Monto año 2029
    """
    try:
        # Esperar a que la tabla cargue completamente
        time.sleep(2)
        
        # Obtener la tabla
        tabla = driver.find_element(By.ID, "tblResultado")
        tbody = tabla.find_element(By.TAG_NAME, "tbody")
        fila = tbody.find_element(By.TAG_NAME, "tr")
        celdas = fila.find_elements(By.TAG_NAME, "td")
        
        # Validar que tenga suficientes columnas
        if len(celdas) < 16:
            print(f"⚠️ Solo {len(celdas)} columnas (se esperaban 16+)", end=" ")
            return None
        
        # Extraer datos por índice de columna
        datos = {
            'opmi': celdas[3].text.strip() if celdas[3].text.strip() else 'NO DISPONIBLE',
            'monto_2026': celdas[12].text.strip() if celdas[12].text.strip() else 'NO DISPONIBLE',
            'monto_2027': celdas[13].text.strip() if celdas[13].text.strip() else 'NO DISPONIBLE',
            'monto_2028': celdas[14].text.strip() if celdas[14].text.strip() else 'NO DISPONIBLE',
            'monto_2029': celdas[15].text.strip() if celdas[15].text.strip() else 'NO DISPONIBLE'
        }
        
        # Verificar que al menos se obtuvo OPMI
        if datos['opmi'] == 'NO DISPONIBLE':
            print(f"⚠️ OPMI vacío", end=" ")
            return None
        
        # Validar que tenga al menos un monto válido
        montos_validos = sum(1 for k, v in datos.items() if k.startswith('monto') and v != 'NO DISPONIBLE' and v != '0.0')
        
        if montos_validos == 0:
            print(f"⚠️ Sin montos válidos", end=" ")
            # Aún así retornamos los datos con OPMI
        
        return datos

    except Exception as e:
        print(f"⚠️ Error extracción: {str(e)[:40]}", end=" ")
        return None

# ================================================================
# PROCESAR CUI
# ================================================================
def procesar_cui(cui, intento=1):
    try:
        url = f"https://ofi5.mef.gob.pe/invierte/pmi/consultapmi?cui={cui}"
        driver.get(url)
        
        Wait(driver, TIMEOUT_ELEMENTO).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(1.5)
        
        # PRIMERO: Verificar si el CUI existe
        if not cui_existe_en_pmi(driver):
            return None, "CUI_NO_EXISTE"

        # SEGUNDO: Intentar extraer datos
        datos = extraer_datos_pmi(driver)
        
        if not datos:
            if intento < MAX_REINTENTOS:
                print(f"🔄{intento+1}...", end=" ")
                time.sleep(1)
                return procesar_cui(cui, intento + 1)
            else:
                return None, "ERROR_EXTRACCION"
        
        return datos, "EXITO"

    except Exception as e:
        if intento < MAX_REINTENTOS:
            print(f"🔄{intento+1}...", end=" ")
            time.sleep(1)
            return procesar_cui(cui, intento + 1)
        else:
            return None, "ERROR_CONEXION"

# ================================================================
# ACTUALIZAR O AGREGAR RESULTADO
# ================================================================
def actualizar_resultado(cui, datos):
    """Actualiza un CUI existente o agrega uno nuevo"""
    for i, r in enumerate(resultados):
        if str(r['CUI']) == str(cui):
            resultados[i] = {
                "CUI": cui,
                "OPMI": datos['opmi'],
                "Monto año 2026": datos['monto_2026'],
                "Monto año 2027": datos['monto_2027'],
                "Monto año 2028": datos['monto_2028'],
                "Monto año 2029": datos['monto_2029']
            }
            return
    
    resultados.append({
        "CUI": cui,
        "OPMI": datos['opmi'],
        "Monto año 2026": datos['monto_2026'],
        "Monto año 2027": datos['monto_2027'],
        "Monto año 2028": datos['monto_2028'],
        "Monto año 2029": datos['monto_2029']
    })

def agregar_fallo(cui):
    """Agrega un CUI como fallido"""
    for r in resultados:
        if str(r['CUI']) == str(cui):
            return
    
    resultados.append({
        "CUI": cui,
        "OPMI": "NO DISPONIBLE",
        "Monto año 2026": "NO DISPONIBLE",
        "Monto año 2027": "NO DISPONIBLE",
        "Monto año 2028": "NO DISPONIBLE",
        "Monto año 2029": "NO DISPONIBLE"
    })

# ================================================================
# LOOP PRINCIPAL
# ================================================================
tiempo_inicio = time.time()
exitosos = 0
fallos_no_existe = 0
fallos_otros = 0

print(f"{'='*60}")
print(f"Inicio: {time.strftime('%H:%M:%S')}")
print(f"{'='*60}\n")

try:
    for idx, cui in enumerate(lista_cuis, 1):
        t_inicio = time.time()
        
        print(f"🆕 [{idx}/{len(lista_cuis)}] {cui}:", end=" ")
        
        try:
            datos, estado = procesar_cui(cui)
            
            if estado == "EXITO":
                actualizar_resultado(cui, datos)
                exitosos += 1
                # Mostrar resumen del dato extraído
                opmi_corto = datos['opmi'][:30] + "..." if len(datos['opmi']) > 30 else datos['opmi']
                print(f"✅ {opmi_corto} ({time.time()-t_inicio:.1f}s)")
                
            elif estado == "CUI_NO_EXISTE":
                agregar_fallo(cui)
                fallos_no_existe += 1
                print(f"⚪ No existe en PMI ({time.time()-t_inicio:.1f}s)")
                
            else:
                agregar_fallo(cui)
                fallos_otros += 1
                print(f"❌ {estado} ({time.time()-t_inicio:.1f}s)")
            
        except Exception as e:
            agregar_fallo(cui)
            fallos_otros += 1
            print(f"❌ Error: {str(e)[:30]} ({time.time()-t_inicio:.1f}s)")

        # GUARDAR CADA N CUIs
        if (idx % GUARDAR_CADA == 0) or (idx == len(lista_cuis)):
            guardar(resultados)
            if idx % GUARDAR_CADA == 0:
                print(f"   💾 Guardado")

        # Progreso cada 10 CUIs
        if idx % 10 == 0:
            t = time.time() - tiempo_inicio
            prom = t / idx
            eta = (len(lista_cuis) - idx) * prom
            total_fallos = fallos_no_existe + fallos_otros
            print(f"\n   📊 ✅{exitosos} ⚪{fallos_no_existe} ❌{fallos_otros} | {t:.0f}s | ETA:{eta:.0f}s\n")

except KeyboardInterrupt:
    print("\n\n⚠️ INTERRUMPIDO")
    guardar(resultados)
    driver.quit()
    print(f"💾 Progreso guardado. Ejecuta nuevamente para continuar.")
    exit()

# ================================================================
# FINALIZAR
# ================================================================
driver.quit()
guardar(resultados)

tiempo_total = time.time() - tiempo_inicio

print(f"\n{'='*60}")
print("🏁 COMPLETADO")
print(f"{'='*60}")
print(f"📊 Total procesados en sesión: {len(lista_cuis)}")
print(f"✅ Exitosos: {exitosos}")
print(f"⚪ CUI no existe en PMI: {fallos_no_existe}")
print(f"❌ Otros errores: {fallos_otros}")
print(f"⏱️ Tiempo: {tiempo_total:.1f}s ({tiempo_total/60:.1f} min)")
print(f"💾 Archivo: {RUTA_SALIDA}")

# Estadísticas finales del archivo completo
df_final = pd.DataFrame(resultados)
total_exitosos = len(df_final[df_final['OPMI'] != 'NO DISPONIBLE'])
total_fallidos = len(df_final[df_final['OPMI'] == 'NO DISPONIBLE'])

print(f"\n📊 ESTADÍSTICAS TOTALES DEL ARCHIVO:")
print(f"   Total CUIs: {len(df_final)}")
print(f"   ✅ Con datos: {total_exitosos} ({total_exitosos/len(df_final)*100:.1f}%)")
print(f"   ⚪ Sin datos (no existen): {total_fallidos} ({total_fallidos/len(df_final)*100:.1f}%)")

# Análisis de montos
if total_exitosos > 0:
    print(f"\n📊 ANÁLISIS DE MONTOS:")
    for anio in ['2026', '2027', '2028', '2029']:
        col = f'Monto año {anio}'
        con_monto = len(df_final[(df_final[col] != 'NO DISPONIBLE') & (df_final[col] != '0.0')])
        print(f"   • {anio}: {con_monto} CUIs con monto ({con_monto/total_exitosos*100:.1f}%)")

print(f"{'='*60}")